In [1]:
%matplotlib inline
import random
import time 
import torch
import torch.nn as nn
from torch.nn import init
import numpy as np
import pandas as pd
from IPython import display
from collections import OrderedDict
import sys
import d2lzh as d2l
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d

# 5. 卷积神经网络 CNN